In [2]:
import pickle
import numpy as np

In [9]:
# read my pickle
# data = pickle.load(open("individuals_friendships_utilities.p","rb"))
# characteristics, connectivity, utilities = data

g_list = pickle.load(open("g_list.pkl", "rb"))

In [16]:
print(g_list[0])
print(np.sum(g_list[0]))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
305.0


In [15]:
small = np.array([[0,1,1,0],
                 [1,0,1,0],
                 [0,0,0,0],
                 [0,0,0,0]])
print(small)
print(np.sum(small))

[[0 1 1 0]
 [1 0 1 0]
 [0 0 0 0]
 [0 0 0 0]]
4


In [14]:
np.sum(small)

4

In [7]:
DENX2, RCHDEN, RELDEN, PTCMUT, PTCASY, RHO2 = ana_dyads(small)

In [17]:
print(ana_dyads(small))

(0.3333333333333333, 0.5, 0.15, 0.5, 0.5, 0.3333333333333333)


In [5]:
def ana_dyads(connectivity):
    # Calculate:
    # density (DENX2),
    # density at maximum reach (RCHDEN),
    # relative density (RELDEN),
    # proportion symmetric dyads (PTCMUT), 
    # asymmetric dyads (PTCASY), and 
    # mutuality index (RHO2)
    
    nodes = connectivity.shape[0]
    mutual_d = 0
    asym_d = 0
    out_degrees = []
    
    
    #density DENX2
    DENX2 = np.sum(connectivity)/(nodes*(nodes-1))
    
    
    # density at maximum reach RCHDEN
    
    # define the function to tranfer adjacency matrix to reachability matrix  
    # Prints reachability matrix of graph[][] using Floyd Warshall algorithm 
    # function found on https://www.geeksforgeeks.org/transitive-closure-of-a-graph/
    '''reach[][] will be the output matrix that will finally 
    have reachability values. 
    Initialize the solution matrix same as input graph matrix'''
    reach =[i[:] for i in connectivity] 
    '''Add all vertices one by one to the set of intermediate 
    vertices. 
    ---> Before start of a iteration, we have reachability value 
    for all pairs of vertices such that the reachability values 
    consider only the vertices in set  
    {0, 1, 2, .. k-1} as intermediate vertices. 
    ----> After the end of an iteration, vertex no. k is 
    added to the set of intermediate vertices and the  
    set becomes {0, 1, 2, .. k}'''
    for k in range(nodes): 
              
        # Pick all vertices as source one by one 
        for i in range(nodes): 
                  
            # Pick all vertices as destination for the 
            # above picked source 
            for j in range(nodes): 
                      
                # If vertex k is on a path from i to j,  
                    # then make sure that the value of reach[i][j] is 1 
                reach[i][j] = reach[i][j] or (reach[i][k] and reach[k][j]) 
    
    RCHDEN= np.sum(reach)/(nodes*(nodes-1))
    
    
    # relative density RELDEN
    RELDEN = np.sum(connectivity)/(10*nodes)

    
    # count out_degree connections per individual and mutual and asymmetrix connections whole network
    for i in range(connectivity.shape[0]):
        out_degree = 0
        for j in range(connectivity.shape[1]):
            if connectivity[i][j] == 1:
                out_degree += 1
                if connectivity[j][i] == 1:
                    mutual_d += 0.5
                elif connectivity[j][i] != 1:
                    asym_d += 1
        out_degrees.append(out_degree)
    
    total_d = mutual_d + asym_d
    
    
    # calculate proportion symmetric dyads (PTCMUT) and asymmetric dyads (PTCASY)
    PTCMUT = mutual_d / total_d
    PTCASY = asym_d / total_d
    
    
    # calculate mutuality index (RHO2) (according to Katz and Powell’s (1955))
    sum_out_degrees = sum(out_degrees)
    mean_out_degrees = sum_out_degrees / len(out_degrees)
    sum_squares_out = 0
    for i in range(len(out_degrees)):
        sum_squares_out += (mean_out_degrees - out_degrees[i])**2
    RHO2 = (2*(nodes - 1)**2 * mutual_d - sum_out_degrees**2 + sum_squares_out) / (sum_out_degrees*(nodes - 1)**2 - sum_out_degrees**2 + sum_squares_out)

    return DENX2, RCHDEN, RELDEN, PTCMUT, PTCASY, RHO2
    